<a href="https://colab.research.google.com/github/ashishshaji/EEG_Classification_Deeplearning/blob/master/Deapdataset_emotion/Deap_Valance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [129]:
pip install git+https://github.com/forrestbao/pyeeg.git

  Cloning https://github.com/forrestbao/pyeeg.git to /private/var/folders/2w/_k8d4ydx6xq809_sr5s1r7qm0000gn/T/pip-req-build-84vvo9y8
  Running command git clone -q https://github.com/forrestbao/pyeeg.git /private/var/folders/2w/_k8d4ydx6xq809_sr5s1r7qm0000gn/T/pip-req-build-84vvo9y8
  Created wheel for pyeeg: filename=pyeeg-0.4.4-py2.py3-none-any.whl size=28121 sha256=ebc3292f903a4eb9319eb02e60936f77dc0ace0648c30f1c7cef12ca8ae77fd0
  Stored in directory: /private/var/folders/2w/_k8d4ydx6xq809_sr5s1r7qm0000gn/T/pip-ephem-wheel-cache-z39oqi0w/wheels/2d/3f/ad/106d4fc80b61d1ea1fc18e76e7439fd98aa043d83d58eae741
Successfully built pyeeg
Note: you may need to restart the kernel to use updated packages.


In [130]:


print("hai")

hai


In [131]:
import os
cwd = os.getcwd()
cwd
os.chdir('/Users/aaliyah/Desktop/biocomp/')
cwd = os.getcwd()
cwd

'/Users/aaliyah/Desktop/biocomp'

In [132]:
 import numpy as np

import pickle as pickle
import pandas as pd
import pyeeg as pe
import pickle as cPickle

In [133]:
channel = [1,2,3,4,6,11,13,17,19,20,21,25,29,31] #14 Channels chosen to fit Emotiv Epoch+
band = [4,8,12,16,25,45] #5 bands
window_size = 256 #Averaging band power of 2 sec
step_size = 16 #Each 0.125 sec update once
sample_rate = 70 #Sampling rate of 128 Hz
subjectList = ['01','02','03']
#List of subjects

In [134]:
from sklearn.preprocessing import normalize


In [143]:
def FFT_Processing (sub, channel, band, window_size, step_size, sample_rate):
    '''
    arguments:  string subject
                list channel indice
                list band
                int window size for FFT
                int step size for FFT
                int sample rate for FFT
    return:     void
    '''
    meta = []
    with open('DEAP/data_preprocessed_python/s' + sub + '.dat', 'rb') as file:

        subject = pickle.load(file, encoding='latin1') #resolve the python 2 data problem by encoding : latin1
        for i in range (0,39):
            # loop over 0-39 trails
            data = subject["data"][i]
            labels = subject["labels"][i]
            start = 0;

            while start + window_size < data.shape[1]:
                meta_array = []
                meta_data = [] #meta vector for analysis
                for j in channel:
                    X = data[j][start : start + window_size] #Slice raw data over 2 sec, at interval of 0.125 sec
                    Y = pe.bin_power(X, band, sample_rate) #FFT over 2 sec of channel j, in seq of theta, alpha, low beta, high beta, gamma
                    meta_data = meta_data + list(Y[0])

                meta_array.append(np.array(meta_data))
                meta_array.append(labels)

                meta.append(np.array(meta_array))    
                start = start + step_size
                
        meta = np.array(meta)
        print('out/s' + sub)
        np.save('out/s' + sub, meta, allow_pickle=True, fix_imports=True)


In [144]:
for subjects in subjectList:
    FFT_Processing (subjects, channel, band, window_size, step_size, sample_rate)

out/s01
out/s02
out/s03


In [146]:
data_training = []
label_training = []
data_testing = []
label_testing = []
data_validation = []
label_validation = []

for subjects in subjectList:
  

    with open('out/s' + subjects + '.npy', 'rb') as file:
        sub = np.load(file,allow_pickle=True)
        for i in range (0,sub.shape[0]):
            if i % 8 == 0 or i % 8== 0:
                data_testing.append(sub[i][0])
                label_testing.append(sub[i][1])
           
            else:
                data_training.append(sub[i][0])
                label_training.append(sub[i][1])

np.save('out/data_training', np.array(data_training), allow_pickle=True, fix_imports=True)
np.save('out/label_training', np.array(label_training), allow_pickle=True, fix_imports=True)
print("training dataset:", np.array(data_training).shape, np.array(label_training).shape)


np.save('out/data_testing', np.array(data_testing), allow_pickle=True, fix_imports=True)
np.save('out/label_testing', np.array(label_testing), allow_pickle=True, fix_imports=True)
print("testing dataset:", np.array(data_testing).shape, np.array(label_testing).shape)
"""
np.save('out/data_validation', np.array(data_validation), allow_pickle=True, fix_imports=True)
np.save('out/label_validation', np.array(label_validation), allow_pickle=True, fix_imports=True)
print("validation dataset:", np.array(data_validation).shape, np.array(label_validation).shape)

"""

training dataset: (49959, 70) (49959, 4)
testing dataset: (7137, 70) (7137, 4)


'\nnp.save(\'out/data_validation\', np.array(data_validation), allow_pickle=True, fix_imports=True)\nnp.save(\'out/label_validation\', np.array(label_validation), allow_pickle=True, fix_imports=True)\nprint("validation dataset:", np.array(data_validation).shape, np.array(label_validation).shape)\n\n'

In [147]:
data = []
label = []
for subjects in subjectList:
  

    with open('out/s' + subjects + '.npy', 'rb') as file:
        sub = np.load(file,allow_pickle=True)
        for i in range (0,sub.shape[0]):
            data.append(sub[i][0])
            label.append(sub[i][1])
np.save('data', np.array(data), allow_pickle=True, fix_imports=True)
np.save('label', np.array(label), allow_pickle=True, fix_imports=True)

In [148]:
df=pd.DataFrame(data=data)
df.to_csv("data.csv",index=False)

df1=pd.DataFrame(data=label)
df1.to_csv("label.csv",index=False)


In [149]:

data1=pd.read_csv("data.csv")

In [150]:
data1


,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,69
0,1467.141307,957.354712,975.500368,886.561347,257.611040,1729.379210,1079.293596,1017.848244,950.011987,277.194922,...,1616.579471,1015.132346,769.122057,991.446909,286.112522,1897.870710,1178.996354,740.014784,722.807957,218.773480
1,1402.080988,872.471171,1069.522141,1008.312775,741.738698,1642.212773,959.802426,1016.890625,1016.753121,486.161277,...,1639.170401,924.285483,828.084355,971.772255,189.171448,1902.103188,1122.875330,761.432035,768.601395,359.695638
2,1569.859186,895.845443,1035.124465,918.612673,113.293654,1678.108287,964.516236,989.247998,968.499790,138.022717,...,1546.708333,986.628413,876.599937,997.347209,244.353918,2057.569881,1161.216557,736.055635,768.684543,240.153545
3,1497.247865,851.437981,959.506731,882.725062,260.333692,1537.752623,911.636878,936.074910,956.197358,383.036705,...,1452.084801,914.005947,814.784673,1033.127883,229.448013,1989.591808,1069.121464,686.572034,697.596546,50.708541
4,1314.476209,919.669964,983.308354,873.276553,352.814075,1351.133996,1007.996757,1000.436238,982.752578,241.910344,...,1433.935229,901.104793,794.645651,1000.369776,199.378771,1739.514883,1011.118559,824.067509,737.803041,496.043021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57091,2941.466671,1667.990021,803.598803,1727.582956,223.909796,1904.887696,1012.261170,516.610274,882.058539,161.676784,...,2627.995444,1496.161376,917.761482,1594.510611,326.740728,2422.375347,1534.229437,898.993497,1709.510794,644.181206
57092,3049.738693,1498.095234,709.837644,1641.664859,126.113945,1926.276835,977.730550,478.385776,835.131073,243.453441,...,2697.377552,1423.471864,994.804976,1580.707942,692.703520,2381.595408,1343.168243,1004.720599,1693.348616,225.530345
57093,2964.787119,1472.638330,636.652963,1655.984762,617.598199,1899.897030,978.312992,485.249539,796.081759,65.974175,...,2678.347948,1383.908050,950.089451,1522.653417,189.358511,2321.112019,1227.066058,909.301931,1690.580232,312.744441
57094,2970.340973,1545.754969,630.235332,1581.842738,328.654513,1850.154481,1026.533202,556.135042,826.079311,140.041788,...,2648.255510,1455.751825,917.489586,1486.280766,191.773088,2354.550619,1168.893927,971.752502,1694.133270,253.219807


In [151]:
label1=pd.read_csv("label.csv")
label1

,0,1,2,3
0,7.71,7.60,6.90,7.83
1,7.71,7.60,6.90,7.83
2,7.71,7.60,6.90,7.83
3,7.71,7.60,6.90,7.83
4,7.71,7.60,6.90,7.83
...,...,...,...,...
57091,4.33,6.21,5.68,4.62
57092,4.33,6.21,5.68,4.62
57093,4.33,6.21,5.68,4.62
57094,4.33,6.21,5.68,4.62


In [152]:
x=data1.values

In [153]:
label1.isnull().sum()


0    0
1    0
2    0
3    0
dtype: int64

In [154]:
y_val=label1.loc[:,'0']

In [155]:
y_val.unique()

array([7.71, 8.1 , 8.58, 4.94, 6.96, 8.27, 7.44, 7.32, 4.04, 1.99, 2.99,
       2.71, 1.95, 4.18, 3.17, 6.81, 2.46, 7.23, 7.17, 8.26, 9.  , 7.09,
       8.15, 7.04, 8.86, 7.28, 7.35, 3.88, 1.36, 2.08, 3.03, 2.28, 3.81,
       2.06, 2.9 , 2.31, 3.33, 3.24, 8.01, 6.05, 5.04, 5.  , 4.96, 4.99,
       7.08, 8.94, 6.  , 8.97, 1.  , 4.06, 2.01, 4.87, 5.33, 7.21, 7.55,
       4.69, 6.92, 6.79, 5.45, 7.14, 7.33, 4.45, 7.94, 6.36, 7.91, 7.29,
       7.36, 7.15, 4.56, 7.1 , 8.14, 6.26, 3.65, 3.31, 3.45, 4.67, 6.72,
       5.59, 4.47, 3.87, 4.44, 5.17, 4.05, 4.72, 3.18, 5.01, 4.92, 4.53,
       4.33])

In [156]:
y_val

0        7.71
1        7.71
2        7.71
3        7.71
4        7.71
         ... 
57091    4.33
57092    4.33
57093    4.33
57094    4.33
57095    4.33
Name: 0, Length: 57096, dtype: float64

In [157]:
y_val.values

array([7.71, 7.71, 7.71, ..., 4.33, 4.33, 4.33])

In [158]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x)
x = scaler.transform(x)
x

array([[-0.56079892, -0.6150587 , -0.55497708, ..., -0.41330096,
        -1.32299145, -0.40788506],
       [-0.56646156, -0.64999505, -0.51476463, ..., -0.31285867,
        -1.23765226,  0.07824073],
       [-0.55185869, -0.64037467, -0.52947627, ..., -0.43186851,
        -1.23749731, -0.33413228],
       ...,
       [-0.43044874, -0.40297836, -0.69989972, ...,  0.38061925,
         0.48051812, -0.08372236],
       [-0.42996535, -0.37288503, -0.70264449, ...,  0.67349896,
         0.48713945, -0.28905884],
       [-0.43882585, -0.41927259, -0.70254317, ...,  0.5420319 ,
         0.3558357 , -0.24327566]])

In [160]:
from keras.utils import to_categorical
y = to_categorical(y_val)
y

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [161]:
x

array([[-0.56079892, -0.6150587 , -0.55497708, ..., -0.41330096,
        -1.32299145, -0.40788506],
       [-0.56646156, -0.64999505, -0.51476463, ..., -0.31285867,
        -1.23765226,  0.07824073],
       [-0.55185869, -0.64037467, -0.52947627, ..., -0.43186851,
        -1.23749731, -0.33413228],
       ...,
       [-0.43044874, -0.40297836, -0.69989972, ...,  0.38061925,
         0.48051812, -0.08372236],
       [-0.42996535, -0.37288503, -0.70264449, ...,  0.67349896,
         0.48713945, -0.28905884],
       [-0.43882585, -0.41927259, -0.70254317, ...,  0.5420319 ,
         0.3558357 , -0.24327566]])

In [162]:
x = np.reshape(x, (x.shape[0],1,x.shape[1]))


In [163]:
y_ar.shape

NameError: name 'y_ar' is not defined

In [164]:
y.shape

(57096, 10)

In [165]:
y

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [166]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 4)

In [167]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(45676, 1, 70)
(45676, 10)
(11420, 1, 70)
(11420, 10)


In [168]:
import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM,BatchNormalization,Activation
model = Sequential()
model.add(LSTM(512, batch_input_shape = (None, None, x.shape[2]),return_sequences=True))
#model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(LSTM(256,activation="relu",return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.5))


model.add(LSTM(128,activation="relu",return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(LSTM(64,activation="relu",return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.3))


model.add(LSTM(32,activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.2))


model.add(Dense(10))
model.add(Activation('sigmoid'))

rmsprop =keras.optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-08)
model.compile(loss='mean_squared_error',
                  optimizer=rmsprop,
                  metrics=['accuracy'])
#adam = keras.optimizers.Adam(lr=0.5, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, None, 512)         1193984   
_________________________________________________________________
batch_normalization_1 (Batch (None, None, 512)         2048      
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 512)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, None, 256)         787456    
_________________________________________________________________
batch_normalization_2 (Batch (None, None, 256)         1024      
_________________________________________________________________
dropout_2 (Dropout)          (None, None, 256)         0         
_________________________________________________________________
lstm_3 (LSTM)                (None, None, 128)        

In [170]:
history = model.fit(x_train, y_train, epochs = 10, batch_size=150,validation_data= (x_test, y_test))


Train on 45676 samples, validate on 11420 samples
Epoch 1/10
45676/45676 [==============================] - 35s 768us/step - loss: 0.2419 - accuracy: 0.1626 - val_loss: 0.2033 - val_accuracy: 0.1788
Epoch 2/10
45676/45676 [==============================] - 36s 782us/step - loss: 0.2101 - accuracy: 0.1841 - val_loss: 0.1852 - val_accuracy: 0.2137
Epoch 3/10
45676/45676 [==============================] - 37s 805us/step - loss: 0.1779 - accuracy: 0.1987 - val_loss: 0.1550 - val_accuracy: 0.2124
Epoch 4/10
45676/45676 [==============================] - 37s 804us/step - loss: 0.1479 - accuracy: 0.2111 - val_loss: 0.1283 - val_accuracy: 0.2431
Epoch 5/10
45676/45676 [==============================] - 37s 821us/step - loss: 0.1237 - accuracy: 0.2277 - val_loss: 0.1108 - val_accuracy: 0.2533
Epoch 6/10
45676/45676 [==============================] - 36s 780us/step - loss: 0.1069 - accuracy: 0.2493 - val_loss: 0.0974 - val_accuracy: 0.2888
Epoch 7/10
45676/45676 [==============================] 

In [171]:
 import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM
model = Sequential()
model.add(LSTM(100, batch_input_shape = (None, None, x.shape[2]),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dense(10, activation='softmax'))


rmsprop =keras.optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-08)
model.compile(loss='mean_squared_error',
                  optimizer=rmsprop,
                  metrics=['accuracy'])
#adam = keras.optimizers.Adam(lr=0.5, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, None, 100)         68400     
_________________________________________________________________
dropout_6 (Dropout)          (None, None, 100)         0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                510       
Total params: 99,110
Trainable params: 99,110
Non-trainable params: 0
_________________________________________________________________
